# Process data (for future loading)

In [ ]:
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
import pickle
import numpy as np
# pip install pykml
# pip install fastkml
# import sweat
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.discrete.discrete_model as dm
import statsmodels.formula.api as smf
from statsmodels.discrete.conditional_models import ConditionalPoisson
from statsmodels.discrete.conditional_models import ConditionalLogit
from statsmodels.regression.mixed_linear_model import MixedLM
import pykml
import overpass
from pykml import parser
from math import radians, sin, cos, sqrt, atan2
from geopy.distance import geodesic
from fastkml import kml
from datetime import datetime, timedelta
from shapely.geometry import Point
import pytz
import matplotlib.pyplot as plt
from gpxcsv import gpxtolist
import seaborn as sns

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import gpxpy
import os
from datetime import datetime, time

# import tcxparser

In [2]:
# Assuming your files are in the specified directory
directory = '\\\\files.drexel.edu\\colleges\\SOPH\\Shared\\UHC\\Projects\\Wellcome_Trust\\Manuscripts\\MS252_Hsu\\Data Build\\Prelim Derived Data\\'
file_pattern = re.compile(r'c(\d+).sas7bdat')

# List all files in the specified directory with the specified pattern
files = [file for file in os.listdir(directory) if file_pattern.match(file)]

In [3]:
# Function to extract the number from the filename
def extract_number(filename):
    match = file_pattern.match(filename)
    if match:
        return int(match.group(1))
    else:
        return None

# Read each file into a DataFrame and name it accordingly
dataframes = {}
for file in files:
    number = extract_number(file)
    if number is not None:
        # Provide the full path to the file
        full_path = os.path.join(directory, file)
        dataframes[number] = pd.read_sas(full_path)

def strata(df):
    df['year'] = df['allDate'].dt.year.astype(str)
    df['year-month'] = df['allDate'].dt.year.astype(str) + '-' + df['allDate'].dt.month.astype(str)
    df['doy'] = df['allDate'].dt.dayofyear  # Day of the year
    df['dow'] = df['allDate'].dt.day_name()  # Day of the week
    df['year-month-dow'] = df['year-month'] + '-' + df['dow'].astype(str)
    return df

In [4]:
def custom_summary_functions(group):
    # Summarize deaths based on gender and age categories
    male_deaths = group[group['male'] == 1]['median_road'].sum()
    female_deaths = group[group['male'] == 0]['median_road'].sum()
    deaths_under_9 = group[group['age_cat'] == b'<=9']['median_road'].sum()
    deaths_10_19 = group[group['age_cat'] == b'10-19']['median_road'].sum()
    deaths_20_34 = group[group['age_cat'] == b'20-34']['median_road'].sum()
    deaths_35_64 = group[group['age_cat'] == b'35-64']['median_road'].sum()
    deaths_65_plus = group[group['age_cat'] == b'65+']['median_road'].sum()

    return pd.Series({
        'country': group['country'].iloc[0],
        'city_size': group['city_size'].iloc[0],
#         'pop_count': group['pop_count'].iloc[0],
        'salid1': group['salid1'].iloc[0],
        'deaths': group['deaths'].sum(),
        'median_road_round': group['median_road'].sum(),
        'median_vehicle_round': group['median_vehicle'].sum(),
        'median_motorcycle_round': group['median_motorcycle'].sum(),
        'median_bicycle_round': group['median_bicycle'].sum(),
        'median_ped_round': group['median_ped'].sum(),
        'L1ADtemp_pw': group['L1ADtemp_pw'].mean(),
        'L1ADtemp_x': group['L1ADtemp_x'].mean(),
        'tmp_pw_percentile': group['tmp_pw_percentile'].mean(),
        'tmp_x_percentile': group['tmp_x_percentile'].mean(),
        'male_deaths': male_deaths,
        'female_deaths': female_deaths,
        'deaths_under_9': deaths_under_9,
        'deaths_10_19': deaths_10_19,
        'deaths_20_34': deaths_20_34,
        'deaths_35_64': deaths_35_64,
        'deaths_65_plus': deaths_65_plus
    })

# Define a dictionary to store the resulting dataframes
result_dataframes = {}

# Iterate through dataframes and apply custom summary functions
for number, df in dataframes.items():
    df['allDate'] = pd.to_datetime(df['allDate'])  # Ensure allDate is in datetime format
    grouped_df = df.groupby('allDate').apply(custom_summary_functions).reset_index()
    result_dataframes[number] = strata(grouped_df)

# Now result_dataframes contains the processed DataFrames with additional death categories

KeyError: 'L1ADtemp_pw'

In [ ]:

# Define the path where you want to save the result_dataframes dictionary
# desktop_path = '/path/to/your/desktop'  # Replace with the actual path to your desktop
result_dataframes_path = os.path.join('C:\\Users\\dpw48\\OneDrive - Drexel University\\code\\MS252-Derek\\Data\\Python Processed Data\\','result_dataframes_subgroup.pkl')

# Save the entire result_dataframes dictionary using pickle
with open(result_dataframes_path, 'wb') as file:
    pickle.dump(result_dataframes, file)

# Call data (pkl)

In [ ]:
result_dataframes_path = os.path.join('Data/MS 252 data_July 2024/Python Processed Data/','result_dataframes_subgroup.pkl')
with open(result_dataframes_path, 'rb') as file:
    loaded_result_dataframes = pickle.load(file)

# Process data, combining all data

In [ ]:
df = pd.concat(loaded_result_dataframes.values(), keys=loaded_result_dataframes.keys()).reset_index()


In [ ]:
csv_path = os.path.join('Data/MS 252 data_July 2024/Python Processed Data/','combined_subroup.csv')
df.to_csv(csv_path)

# Process data, combining all city-level effect modifiers

In [ ]:
BEC1 = pd.read_csv(os.path.join('Data/MS252 requested data/','BEC_L1AD_08162023.csv'))
BEC1 = BEC1[['ISO2','SALID1','BECADSTTLGAVGL1AD','BECCZL1AD','BECADSTTDENSL1AD','BECADLRDENSL1AD','BECADINTDENSL1AD','BECPTCHDENSL1AD','BECADINTDENS3L1AD','BECADINTDENS4L1AD','BECADSTTPNODEAVGL1AD','BECADSTTPNODESDL1AD','BECADSTTLGAVGL1AD','BECADCRCTYAVGL1AD','BECSTTPL1AD','BECPCTURBANL1AD','BECGSPCTL1AD', 'BECGSPTCHDENSL1AD','BECMINWAGEL1AD','BECELEVATIONMAXL1AD','BECELEVATIOVEL1AD','BECELEVATIONMEDIANL1AD','BECELEVATIONMINL1AD','BECELEVATIONP25L1AD','BECELEVATIONP75L1AD','BECELEVATIONSTDL1AD','BECSLOPEMAXL1AD','BECSLOPEAVEL1AD','BECSLOPEMEDIANL1AD','BECSLOPEMINL1AD','BECSLOPEP25L1AD','BECSLOPEP75L1AD','BECSLOPESTDL1AD']]
BEC2 = pd.read_csv(os.path.join('Data/MS252 requested data/','BEC_RESTRICTED_L1AD_08162023.csv'))
BEC2 = BEC2[['SALID1','BECURBTRVDELAYINDEXL1AD','BECURBAVGTRAFTIMEL1AD','BECURBTRVDELAYTIMEL1AD','BECPARKPCTAREAL1AD']]
BEC = pd.merge(BEC1, BEC2, on='SALID1', how='inner')

In [ ]:
temp = pd.read_csv(os.path.join('Data/MS252 requested data/','SEC_INDEXSCORES_L1AD_07102023.csv'))
latest_indices = temp.groupby('SALID1')['YEAR'].idxmax()
SEC = temp.loc[latest_indices, ['SALID1','CNSSEI_L1AD', 'CNSSE1_L1AD', 'CNSSE2_L1AD','CNSSE3_L1AD']]

In [ ]:
temp = pd.read_csv(os.path.join('Data/MS252 requested data/','VehicleRegistration_L1AD_20201027.csv'))
latest_indices = temp.groupby('SALID1')['YEAR'].idxmax()
REG = temp.loc[latest_indices, ['SALID1','BECMTRBRATEL1AD','BECPAVRATEL1AD','BECTOTVRRATEL1AD']]

In [ ]:
df['salid1']=df['salid1'].astype(int)
df_all=df.merge(BEC, left_on='salid1', right_on='SALID1').merge(SEC, on='SALID1').merge(REG, how='left',on='SALID1')
#careful about this fillna
# df_all.fillna(df_all.mean(), inplace=True)

In [ ]:
df_all.to_csv(os.path.join('Data/MS 252 data_July 2024/Python Processed Data/','final_subgroup.csv'), index=False)



# load data back

In [ ]:
final=pd.read_csv('/Users/cheng-kaihsu/Library/Mobile Documents/com~apple~CloudDocs/Berkeley/Fall 2023/SALURBAL/Data/MS 252 data_July 2024/Python Processed Data/final_subgroup.csv')

################################################################################
# BELOW NOT NEEDED ANYMORE SINCE THE NEW DATASET ALREADY CONTAINS ROUNDED VALUES
################################################################################
# Function to round non-integer values randomly up or down
# def random_round(value):
#     if value.is_integer():
#         return int(value)
#     else:
#         return int(value) + np.random.choice([0, 1])
# #
# # Apply the function to the 'median_road' column
# final['median_road_round'] = final['median_road'].apply(random_round)
# final['median_vehicle_round'] = final['median_vehicle'].apply(random_round)
# final['median_motorcycle_round'] = final['median_motorcycle'].apply(random_round)
# final['median_bicycle_round'] = final['median_bicycle'].apply(random_round)
# final['median_ped_round'] = final['median_ped'].apply(random_round)
# final['male_deaths_round'] = final['male_deaths'].apply(random_round)
# final['female_deaths_round'] = final['female_deaths'].apply(random_round)
# final['deaths_under_9_round'] = final['deaths_under_9'].apply(random_round)
# final['deaths_10_19_round'] = final['deaths_10_19'].apply(random_round)
# final['deaths_20_34_round'] = final['deaths_20_34'].apply(random_round)
# final['deaths_35_64_round'] = final['deaths_35_64'].apply(random_round)
# final['deaths_65_plus_round'] = final['deaths_65_plus'].apply(random_round)


# Temperature cluster

In [ ]:
# pip install pyreadstat
import pyreadstat

# Replace "path/to/your/file.sas7bdat" with the actual file path
file_path = "/Users/cheng-kaihsu/Library/Mobile Documents/com~apple~CloudDocs/Berkeley/Fall 2023/SALURBAL/Data/city_level_temp_w_clusters.sas7bdat"

# Read the SAS dataset file
temp_cluster, meta = pyreadstat.read_sas7bdat(file_path)


In [ ]:
final_temp_cluster=final.merge(temp_cluster, left_on='salid1', right_on='nsalid1')

In [ ]:
final_temp_cluster.to_csv(os.path.join('Data/MS 252 data_July 2024/Python Processed Data/','final_temp_cluster_subgroup.csv'), index=False)


# Descriptive

In [ ]:
temp=pd.read_csv('Data/MS 252 data_July 2024/Python Processed Data/final_temp_cluster_subgroup.csv')

In [ ]:
temp.groupby(['country','year'])['SALID1'].nunique().reset_index(name='unique_SALID1_count').to_csv('Data/MS 252 data_July 2024/Python Processed Data/SALID_count_by_country_year.csv')


In [ ]:
# Set pandas options to display floats in a readable format
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

# Calculate descriptive statistics
temp.describe()[['median_road_round', 'median_vehicle_round',
       'median_motorcycle_round', 'median_bicycle_round', 'median_ped_round','male_deaths', 'female_deaths', 'deaths_under_9', 'deaths_10_19',
       'deaths_20_34', 'deaths_35_64', 'deaths_65_plus','L1ADtemp_pw', 'tmp_pw_percentile']].to_csv('Data/MS 252 data_July 2024/Python Processed Data/des.csv')


In [ ]:
temp.columns[:40]

In [ ]:
city_counts = temp.groupby(['cluster_ward_std_6', 'salid1']).size().reset_index(name='counts')
city_counts.groupby('cluster_ward_std_6')['salid1'].nunique().reset_index(name='num_cities')


In [ ]:
temp['country'].unique()

In [ ]:
temp['salid1'].nunique()